In [7]:
import os
import pandas as pd
from pandas import DataFrame
import numpy as np
from Electric_Vehicle_Prediction.configurations.mongo_db_connection import mongodb_client
from evidently import Report
from evidently.presets import DataDriftPreset, RegressionPreset
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option("display.max_columns", None)
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from Electric_Vehicle_Prediction.constants import (
    MONGODB_DATABASE_NAME,
    MONGODB_COLLECTION_NAME,
)

database = mongodb_client[MONGODB_DATABASE_NAME]
collection = database[MONGODB_COLLECTION_NAME]
data = collection.find({}, {"_id": 0, "index": 0})
df = pd.DataFrame(list(data))
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,2T3YL4DV0E,King,Bellevue,WA,98005.0,2014,TOYOTA,RAV4,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,103.0,0.0,41.0,186450183,POINT (-122.1621 47.64441),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
1,5YJ3E1EB6K,King,Bothell,WA,98011.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,0.0,1.0,478093654,POINT (-122.20563 47.76144),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
2,5UX43EU02S,Thurston,Olympia,WA,98502.0,2025,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,40.0,0.0,35.0,274800718,POINT (-122.92333 47.03779),PUGET SOUND ENERGY INC,5.306701e+10
3,JTMAB3FV5R,Thurston,Olympia,WA,98513.0,2024,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,42.0,0.0,2.0,260758165,POINT (-122.81754 46.98876),PUGET SOUND ENERGY INC,5.306701e+10
4,5YJYGDEE8M,Yakima,Selah,WA,98942.0,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,15.0,236581355,POINT (-120.53145 46.65405),PACIFICORP,5.307700e+10


In [3]:
# List of columns to drop
drop_columns = [
    "VIN (1-10)",
    "Postal Code", 
    "Legislative District",
    "DOL Vehicle ID", 
    "2020 Census Tract", 
    "County", 
    "State", 
    "City", 
    "Electric Utility", 
    "Vehicle Location"
]

# Drop the specified columns
data = df.drop(columns=drop_columns)

In [5]:
data.dropna(inplace=True)

In [ ]:
def detect_dataset_drift(reference_df: DataFrame,current_df: DataFrame,) -> bool:
    report = Report(metrics=[DataDriftPreset()])
    report.run(reference_data=reference_df, current_data=current_df)
    json_report = report
    return json_report


In [30]:
json_report = detect_dataset_drift(data[:200000], data[200000:])
print(json_report)

{'metrics': [DataDriftPreset(type='evidently:metric_container:DataDriftPreset', include_tests=True, columns=None, embeddings=None, embeddings_drift_method=None, drift_share=0.5, method=None, cat_method=None, num_method=None, text_method=None, per_column_method=None, threshold=None, cat_threshold=None, num_threshold=None, text_threshold=None, per_column_threshold=None)], 'metadata': {}, 'tags': [], '_timestamp': None, 'include_tests': False}


In [32]:
json_report["metrics"][0]

DataDriftPreset(type='evidently:metric_container:DataDriftPreset', include_tests=True, columns=None, embeddings=None, embeddings_drift_method=None, drift_share=0.5, method=None, cat_method=None, num_method=None, text_method=None, per_column_method=None, threshold=None, cat_threshold=None, num_threshold=None, text_threshold=None, per_column_threshold=None)

In [ ]:
from evidently import Report
from evidently.presets import DataDriftPreset, RegressionPreset

In [ ]:
column_mapping = ColumnMapping()
column_mapping.target = "MEDV"

boston_data_drift_dashboard = Dashboard(tabs=[DataDriftTab(), CatTargetDriftTab()])
boston_data_drift_dashboard.calculate(
    data[:200], 
    data[200:], 
    column_mapping=column_mapping
)
boston_data_drift_dashboard.show()

In [10]:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp

# Example: Generate sample datasets
reference_data = df.iloc[:120000]  # Reference dataset (e.g., older data)
current_data = df.iloc[120000:]  # Current dataset (e.g., new data)

# Data drift detection function
def detect_data_drift(reference, current, threshold=0.05):
    drift_results = {}
    for column in reference.columns:
        # Perform KS test for numerical features
        if np.issubdtype(reference[column].dtype, np.number):
            stat, p_value = ks_2samp(reference[column], current[column])
            drift_results[column] = {"p_value": p_value, "drift_detected": p_value < threshold}
        else:
            # Handle categorical data if needed
            drift_results[column] = "Categorical column: Not tested"
    return drift_results

# Perform data drift detection
drift_results = detect_data_drift(reference_data, current_data)

# Display results
for feature, result in drift_results.items():
    print(f"{feature}: {result}")

VIN (1-10): Categorical column: Not tested
County: Categorical column: Not tested
City: Categorical column: Not tested
State: Categorical column: Not tested
Postal Code: {'p_value': np.float64(nan), 'drift_detected': np.False_}
Model Year: {'p_value': np.float64(0.022156315235406652), 'drift_detected': np.True_}
Make: Categorical column: Not tested
Model: Categorical column: Not tested
Electric Vehicle Type: Categorical column: Not tested
Clean Alternative Fuel Vehicle (CAFV) Eligibility: Categorical column: Not tested
Electric Range: {'p_value': np.float64(nan), 'drift_detected': np.False_}
Base MSRP: {'p_value': np.float64(nan), 'drift_detected': np.False_}
Legislative District: {'p_value': np.float64(nan), 'drift_detected': np.False_}
DOL Vehicle ID: {'p_value': np.float64(8.794436448709497e-11), 'drift_detected': np.True_}
Vehicle Location: Categorical column: Not tested
Electric Utility: Categorical column: Not tested
2020 Census Tract: {'p_value': np.float64(nan), 'drift_detected